<h1><center> DPLL </center></h1>

This is an implementaion of the DPLL algorithm for determining the satisfiability of knowledge bases (KBs) in symbolic logic, with KBs in the form of Conjunctive Normal Form (CNF). It does not convert the KB to CNF, and the input kb has to be of the form:
{-A,B,E}, {-B, A}, {-E, A}, where "," between sets represent "and", and "or" within sets.
The algorithm is called by using DPLLSatisfiable(kb), which then arranges the symbols by order of importance and occurence in the kb. Importance is ranked: pure symbols first, unit clauses next, and other symbols last.

DPLL is then called recursively on the clauses, it removes all satisfied clauses before recursing.
If no clauses are left unsatisfied, i.e. all clauses are satisfied, it returns True, and the model (a dictionary of literals and their assigned truth-value) that satisfied it.
If any clause is contradicted, the model cannot be satisfied with the existing assumptions and is looped back to the previous recursion, and a different assignment is made.
If all assignments been attempted, the DPLL returns False, and a model where all symbols are "Free".

Importing collections to use a default dictionary for counting literals' occurrences.
Importing heapq to always pop the most common literal, so we can implement the degree heuristic, as well as for ordering by importance for the pure-symbol and unit-clause heuristics.
Enum is used for readibility and is not necessary. Used in QueueImportance.

The Literal class is initiated with a name and sign,
the hash function is used for storing literals in dictionries and sets,
the neg uses - signs to store self.sign = False as a negative,
the eq function allows us to store negative and positive literals as identical in sets and dictionaries,
str was used for visual testing, and left if we wish to print the sign as well as the name of the literal.
lt is less-then and is required for tie-breaking if literals occur the same number of times in the same importance.
repr is used to print literals' names instead of their stored value.

In [1]:
from collections import defaultdict
import heapq
from enum import Enum


class QueueImportance(Enum):
    """
    class used for readability for ordering the literals by their importance –
    pure literals before unit clauses before other literals.
    """
    PURE = 1
    UNIT = 2
    REST = 3


class Literal:
    def __init__(self, name, sign=True):
        """
        :param name: string containing the literal name, e.g. A, B, C, etc.
        :param sign: True by default, False if negative
        """
        self.name, self.sign = name, sign
        return

    def __hash__(self):
        """
        A function to allow us to hash and retrieve literals
        makes searching easier, and also possible within sets
        :return: a hash for our literal
        """
        return hash(self.name)

    def __neg__(self):
        """
        to allow negative literals to be allowed in clauses
        :return: returns the same name of the literal but with opposite sign
        """
        return Literal(self.name, not self.sign)

    def __eq__(self, other):
        """
        specify two literals with the same name are the same regardless of sign
        :param other: another literal with the same name
        :return: two identical literal names are treated as a single literal
        """
        return self.name == other.name

    def __str__(self):
        """
        :return: prints the name and sign of the literal
        """
        if self.sign:
            return str(self.name)
        return "-"+str(self.name)

    def __lt__(self, other):
        """
        less-then implemented to resolve heappush instances when
        both literals appear an equal number of times in the kb.
        Used for our degree heuristic
        :param other: another literal
        :return: decides which literal is 'larger'
                    based on their str representation
        """
        return str(self.name) < str(other.name)

    def __repr__(self):
        """
        :return: prints the name instead of the hash of the literal
        """
        return self.name

Our function to check for clause satisfiability.

In [2]:
def clauseSatisfied(clause, model):
    """
    a function to test whether a clause is satisfied with the model
    :param clause: the clause to be tested, in horn format
    :param model: the assigned truth values of all literals in the model
    :return: True if satisfied, False if not and all literals are assigned
                If one of the literals is till "free", returns nothing.
    """
    # a var to determine if any literal is unassigned
    clause_has_free = False

    for literal in clause:
        """
        if any of the literals is true, the clause is True for DPLL
        if none are true, and some are unassigned, the clause is not resolved
        if all are false, returns false. 
        """
        if literal in model:
            """
            if the literal is in the model,
            if it is free, return nothing,
            if it isn't, and sign is True, return True,
            otherwise, return False
            """
            tested_literal = model[literal]
            if tested_literal == "Free":
                clause_has_free = True
            elif literal.sign:
                if tested_literal is True:
                    return True
            else:
                if tested_literal is False:
                    return True

    if clause_has_free:
        return
    return False

The DPLL-Satisfiable function is none-recursive but calss the recursive DPLL.
It handles the kb breakdown into clauses and symbols, and initiates a model with all the symbols set to 'Free'.
Symbols are passed into the DPLL by importance and occurrence, which should allow DPLL to be resolved faster if the model is satisfiable.

In [3]:
def DPLLSatisfiable(kb):
    """
    takes in a kb and returns if it is or isn't satisfiable,
    as well as a model that satisfies it, if one exists.
    :param kb: knowledge base of the form - [{A,B},{A,-C},{-A,B,D}]
    :return:
    satisfiable: boolean value indicating whether
        or not the KB is satisfiable
    model: python dictionary that has a Literal as the key and
        the value being one of the strings ‘true’, ‘false’ or ‘free’,
        depending on whether the literal must be true or false or can
        be freely chosen to ensure satisfiability
        e.g. model = {A: true, B: true, C: false, D: free}
    """
    clauses = kb  # the clauses in CNF

    # a dictionary that stores all our symbols and the number of times
    # they occur. Used for our degree heuristic
    symbols_ranking = defaultdict(lambda: 0)
    pure_symbols = {}
    unit_clauses = set()

    for clause in kb:
        # unit clauses are of len 1, add them to the unit_clauses
        if len(clause) == 1:
            unit_clauses.union(clause)

        # count the occurrences of each literal in kb
        for literal in clause:
            symbols_ranking[literal] += 1
            # pure symbols are tested later if their corresponding set
            # has only one sign.
            if literal not in pure_symbols:
                pure_symbols[literal] = set()
            pure_symbols[literal].add(literal.sign)

    symbols = []

    """
    create a heap to order our symbols by occurrence
    to handle pure symbols, we order by tuple which orders them before
    regular symbols and unit clauses,
    to handle unit_clauses, we order by a tuple which orders them before
    regular symbols.
    """
    for literal, cnt in symbols_ranking.items():
        if len(pure_symbols[literal]) == 1:
            heapq.heappush(symbols, ((QueueImportance.PURE, -cnt), literal))
        elif literal not in unit_clauses:
            heapq.heappush(symbols, ((QueueImportance.REST, -cnt), literal))
        else:
            heapq.heappush(symbols, ((QueueImportance.UNIT, -cnt), literal))

    # create the model, assigning 'Free' to all symbols
    model = {symbol: 'Free' for symbol in symbols_ranking.keys()}

    return DPLL(clauses, symbols, model)

Recursive DPLL passes unsatisfied clauses to recusive calls, if all clauses are satisfied – returns true. If any clause is contradicted – returns to the last decision point and attempts to reassign the contradictory sign.
If no solution is found, returns False, the model cannot be satisfied.

In [4]:
def DPLL(clauses, symbols, model):
    """
    returns True if satisfiable, false if not.
    Also returns the model that found satisfiability.
    :param clauses: a list of the clauses in the kb
    :param symbols: a list of all symbols (literals) in the kb
    :param model: the existing model to expand.
    :return: same as DPLLSatisfiable –
        satisfiable: boolean value indicating whether
            or not the KB is satisfiable
        model: python dictionary that has a Literal as the key and
            the value being one of the strings ‘true’, ‘false’ or ‘free’,
            depending on whether the literal must be true or false or can
            be freely chosen to ensure satisfiability
            e.g. model = {A: true, B: true, C: false, D: free}
    """
    satisfiable = False

    unsatisfied_clauses = []
    for clause in clauses:
        c = clauseSatisfied(clause, model)
        if c is False:
            # check if any clause is false in the model, if so, the model is not satisfiable
            return False, model
        if c is None:
            unsatisfied_clauses.append(clause)
        if not unsatisfied_clauses:
            # if all clauses are satisfied, return True and the model
            return True, model

    """
    if there are still unassigned symbols,
        assign the most common literal to True,
            and continue deepening the search with the unsatisfied clauses
        if it is not satisfiable with this literal as True does
        not satisfy the model, assign it to False and continue searching
        with the unsatisfied clauses. If that too fails, assign it to 'Free'.
    else (i.e. when reaching the bottom of the search and there are no more
        symbols to be assigned), return whether satisfiable and the model
        This should return our default – False.
    
    Since we ordered pure symbols before unit clauses, and unit clauses before
    other symbols, this implements the respective heuristics
    """
    if len(symbols) > 0:
        cnt, P = heapq.heappop(symbols)
        model[P] = True
        satisfiable, model = DPLL(unsatisfied_clauses, symbols, model)

        if not satisfiable:
            model[P] = False
            satisfiable, model = DPLL(unsatisfied_clauses, symbols, model)
            if not satisfiable:
                model[P] = "Free"
                heapq.heappush(symbols, (cnt, P))

    return satisfiable, model


In [ ]:
A = Literal("A")
B = Literal("B")
C = Literal("C")
D = Literal("D")
E = Literal("E")
F = Literal("F")

# example from Russle and Norvig, problem 7.20
s1 = {-A,B,E}, {-B, A}, {-E, A}
s2 = {-E}, {D}
s3 = {-C}, {-F}, {-B}
s4 = {-E}, {B}
s5 = {-B}, {F}
s6 = {-B}, {C}

# since we have to use a list, I just rewrote the clasues, but we could think of recursive solutions instead
kb = [{-A,B,E}, {-B, A}, {-E, A}, {-E}, {D}, {-C}, {-F}, {-B}, {-E}, {B}, {-B}, {F}, {-B}, {C}]
print(DPLLSatisfiable(s1))